In [6]:
import pandas as pd
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [3]:
datadir = '/opt/data'
localdir = '/opt/program'
tripsdf = pd.read_csv(f'{datadir}/2013-07 - Citi Bike trip data.csv')
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)

In [5]:
dict(tripsdf.iloc[0])

{'tripduration': 634,
 'starttime': '2013-07-01 00:00:00',
 'stoptime': '2013-07-01 00:10:34',
 'start station id': 164,
 'start station name': 'E 47 St & 2 Ave',
 'start station latitude': 40.75323098,
 'start station longitude': -73.97032517,
 'end station id': 504,
 'end station name': '1 Ave & E 15 St',
 'end station latitude': 40.73221853,
 'end station longitude': -73.98165557,
 'bikeid': 16950,
 'usertype': 'Customer',
 'birth year': '\\N',
 'gender': 0}

In [7]:
datadir = f'/opt/program/artifacts/2020-07-08T143732Z'
proc_bundle = joblib.load(f'{datadir}/proc_bundle.joblib')

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.20.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [9]:
proc_bundle['proc_bundle'].keys()

dict_keys(['enc', 'usertype_le', 'le'])

In [11]:
record = {
 'starttime': '2013-07-01 00:00:00',
 'start station id': 164,
 'start station name': 'E 47 St & 2 Ave',
 'start station latitude': 40.75323098,
 'start station longitude': -73.97032517,
# unknown
# 'end station id': 504,
# 'end station name': '1 Ave & E 15 St',
# 'end station latitude': 40.73221853,
# 'end station longitude': -73.98165557,
# 'stoptime': '2013-07-01 00:10:34',
# 'tripduration': 634,
 'bikeid': 16950,
 'usertype': 'Customer',
 'birth year': '\\N',
 'gender': 0}

inputdf = pd.DataFrame.from_records([record])

##### Here was my pv1 recipe for preprocess + predict 


```python
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# preproc
(X_transformed,
     one_hot_enc, le,
     y_enc) = pv1.preprocess(X_train, y_train, # X[train_index]
                         neighborhoods)
labels = le.classes_

# Test set
X_test_transformed = one_hot_enc.transform(X_test)
y_test_enc = le.transform(y_test)


# predict
y_prob_vec = fu.predict_proba(X_test_transformed, bundle_loc=bundle_loc)

```

##### ok what if the bundle already exists 
* Actually pv1 doesnt handle existing preproc bundles ..

#### but pv2 does handle existing preproc bundles 
* the output is saved as a svm style file.. 
```python
import fresh.preproc.v2 as pv2

X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)
X_train, X_test, y_train, y_test = train_test_split(X, y)


test_loc = pv2.preprocess(
        X_test, y_test, neighborhoods, proc_bundle=proc_bundle,
        workdir=workdir,
        dataset_name='test')
```

In [13]:
reload(fu)

<module 'fresh.utils' from '/opt/program/fresh/utils.py'>

In [21]:
!pwd

/opt/program


In [23]:
reload(fu)
rootdir = '/opt/program'
workdir = fu.make_work_dir(rootdir)
print(workdir)

/opt/program/artifacts/2020-08-19T144654Z


In [41]:
proc_bundle['proc_bundle'].keys()


dict_keys(['enc', 'usertype_le', 'le'])

In [43]:
# Hmm really weird, I was getting this weird error, 
# AttributeError: 'OneHotEncoder' object has no attribute 'drop'
# And somehow got rid of it by setting 
# enc.drop = None

In [52]:
X = fu.prepare_data(inputdf, stationsdf, labelled=False)
neighborhoods = fu.neighborhoods_from_stations(stationsdf)

print(['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday', ])
print(X)

input_loc = pv2.preprocess(
        X=X, neighborhoods=neighborhoods, 
        proc_bundle=proc_bundle['proc_bundle'],
        workdir=workdir,
        dataset_name='input')
print(input_loc)

  0%|          | 0/1 [00:00<?, ?it/s]

['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday']
[['Midtown East' 0 4 'Customer' 1]]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [51]:
reload(fu)
# num_rows = X.shape[0]
print('num rows', num_rows)
# fu.get_slices(list(range(num_rows)), num_slices=10)
# fu.get_slices([1], num_slices=1) # [[1, 2]]


#fu.get_partitions(vec, slice_size, keep_remainder)

#fu.get_slices([1], num_slices=10) # [[1, 2]]

#fu.get_partitions([1], slice_size=1, keep_remainder=True)


#fu.get_slices(range(100), num_slices=11)
fu.get_slices(list(range(1)), num_slices=10)

X[0:1, :3] # array([['Midtown East', 0, 4]], dtype=object)

proc_bundle['proc_bundle']['enc'].transform(X[0:1, :3]).toarray()
enc = proc_bundle['proc_bundle']['enc']
enc.drop = None
enc.transform(X[0:1, :3]).toarray()

num rows 1


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 1.]])

In [54]:
# Next.. hmm , when trying to preprocess... 
# ValueError: Input contains NaN, infinity or a value too large for dtype('float64').
# - I suspect because I left in the 'y' as None , ... 
# 
#      68         elif filetype == 'libsvm':
# ---> 69             to_libsvm(X_transformed, y_enc, outfile)


# And then Next...  in order to rrun this line...
# y_prob_vec = fu.predict_proba(X_test_transformed, bundle_loc=bundle_loc)
# got to load the bundle_loc ... 
i = 1187
artifactsdir = 'artifacts/2020-07-10T135910Z'
bundle = joblib.load(f'{artifactsdir}/{i}_bundle_with_metrics.joblib')
model = bundle['xgb_model']


In [57]:
from sklearn.datasets import dump_svmlight_file
# dump_svmlight_file  .. can it take no y ? what happens then? 
# in that case i should  instead of dumping to file, just return it , makes more sense anyway. 

In [61]:
reload(fu)

<module 'fresh.utils' from '/opt/program/fresh/utils.py'>

In [63]:
reload(pv2)

<module 'fresh.preproc.v2' from '/opt/program/fresh/preproc/v2.py'>

In [64]:
X = fu.prepare_data(inputdf, stationsdf, labelled=False)
neighborhoods = fu.neighborhoods_from_stations(stationsdf)

print(['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday', ])
print(X)

X_transformed = pv2.preprocess(
        X=X, neighborhoods=neighborhoods, 
        proc_bundle=proc_bundle['proc_bundle'],
        #workdir=workdir,
        #dataset_name='input'
)
print(X_transformed)

  0%|          | 0/1 [00:00<?, ?it/s]

['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday']
[['Midtown East' 0 4 'Customer' 1]]
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0 1]]


In [65]:
#dtrain = xgb.DMatrix(f'{train_loc}?format=libsvm')
dtrain = xgb.DMatrix(X_transformed)
# actuals = dtrain.get_label()

In [66]:
 y_prob_vec = model.predict(dtrain)
predictions = np.argmax(y_prob_vec, axis=1)


In [67]:
y_prob_vec, predictions

(array([[0.00260068, 0.00211338, 0.18238813, 0.04775023, 0.00407627,
         0.03426266, 0.01549505, 0.02768018, 0.00213743, 0.00634305,
         0.02986856, 0.02481377, 0.07334273, 0.07258937, 0.04883923,
         0.00380077, 0.00281027, 0.00450624, 0.03994255, 0.04935879,
         0.00830992, 0.0083217 , 0.00283933, 0.00307513, 0.00572311,
         0.00539479, 0.00300998, 0.0088579 , 0.00437763, 0.00306665,
         0.00585913, 0.00202413, 0.00434052, 0.00435104, 0.00632701,
         0.00504773, 0.00515511, 0.01245811, 0.00278124, 0.02105542,
         0.00562638, 0.01457816, 0.00257211, 0.01134078, 0.03223165,
         0.00336919, 0.00675962, 0.01909184, 0.00364397, 0.0023981 ,
         0.00664525, 0.0740085 , 0.00168374, 0.01895574]], dtype=float32),
 array([2]))

#### ok make a multi bundle from those

In [73]:
# Going from ....
# this proc bundle
datadir = f'/opt/program/artifacts/2020-07-08T143732Z'
proc_bundle = joblib.load(f'{datadir}/proc_bundle.joblib')

# and this model bundle
i = 1187
artifactsdir = 'artifacts/2020-07-10T135910Z'
bundle = joblib.load(f'{artifactsdir}/{i}_bundle_with_metrics.joblib')
# model = bundle['xgb_model']

# and neighborhoods
localdir = '/opt/program'
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)
neighborhoods = fu.neighborhoods_from_stations(stationsdf)

out_loc = f'{workdir}/all_bundle.joblib'
print('Outputing a combined bundle here...', out_loc)
joblib.dump({'notebook': '2020-08-18-glue.ipynb',
            'model_bundle': {
                'original_filename': f'{artifactsdir}/{i}_bundle_with_metrics.joblib',
                'bundle': bundle},
            'proc_bundle': {
                'original_filename': f'{datadir}/proc_bundle.joblib',
                'bundle': proc_bundle
            },
            'neighborhoods_bundle': {
                'original_filename': f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                'neighborhoods': neighborhoods
            } 
            }, out_loc)


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.20.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Outputing a combined bundle here... /opt/program/artifacts/2020-08-19T144654Z/all_bundle.joblib


['/opt/program/artifacts/2020-08-19T144654Z/all_bundle.joblib']

In [81]:
def full_predict(bundle, record):
    inputdf = pd.DataFrame.from_records([record])
    
    X = fu.prepare_data(inputdf, stationsdf, labelled=False)
    
    print(['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday', ])
    print(X)

    X_transformed = pv2.preprocess(
        X=X, 
        neighborhoods=bundle['neighborhoods_bundle']['neighborhoods'], 
        proc_bundle=bundle['proc_bundle']['bundle']['proc_bundle'],
        #workdir=workdir,
        #dataset_name='input'
    )
    print(X_transformed)
    dtrain = xgb.DMatrix(X_transformed)
    
    model = bundle['model_bundle']['bundle']['xgb_model']
    
    y_prob_vec = model.predict(dtrain)
    predictions = np.argmax(y_prob_vec, axis=1)

    return y_prob_vec, predictions
    
all_bundle_loc = '/opt/program/artifacts/2020-08-19T144654Z/all_bundle.joblib'
bundle = joblib.load(all_bundle_loc)

# fix bundle... 
bundle['proc_bundle']['bundle']['proc_bundle']['enc'].drop = None

record = {
 'starttime': '2013-07-01 00:00:00',
 'start station id': 164,
 'start station name': 'E 47 St & 2 Ave',
 'start station latitude': 40.75323098,
 'start station longitude': -73.97032517,
# unknown
# 'end station id': 504,
# 'end station name': '1 Ave & E 15 St',
# 'end station latitude': 40.73221853,
# 'end station longitude': -73.98165557,
# 'stoptime': '2013-07-01 00:10:34',
# 'tripduration': 634,
 'bikeid': 16950,
 'usertype': 'Customer',
 'birth year': '\\N',
 'gender': 0}


full_predict(bundle, record)

  0%|          | 0/1 [00:00<?, ?it/s]

['start_neighborhood', 'gender', 'time_of_day', 'usertype', 'weekday']
[['Midtown East' 0 4 'Customer' 1]]
[[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0 1]]


(array([[0.00260068, 0.00211338, 0.18238813, 0.04775023, 0.00407627,
         0.03426266, 0.01549505, 0.02768018, 0.00213743, 0.00634305,
         0.02986856, 0.02481377, 0.07334273, 0.07258937, 0.04883923,
         0.00380077, 0.00281027, 0.00450624, 0.03994255, 0.04935879,
         0.00830992, 0.0083217 , 0.00283933, 0.00307513, 0.00572311,
         0.00539479, 0.00300998, 0.0088579 , 0.00437763, 0.00306665,
         0.00585913, 0.00202413, 0.00434052, 0.00435104, 0.00632701,
         0.00504773, 0.00515511, 0.01245811, 0.00278124, 0.02105542,
         0.00562638, 0.01457816, 0.00257211, 0.01134078, 0.03223165,
         0.00336919, 0.00675962, 0.01909184, 0.00364397, 0.0023981 ,
         0.00664525, 0.0740085 , 0.00168374, 0.01895574]], dtype=float32),
 array([2]))

In [76]:
# next... ^ finish the all predict func... 
# so close